In [1]:
import requests
import json
import pandas as pd
import datetime
import time
from tqdm import tqdm
import psycopg2
import random
from typing import List
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np

import sqlalchemy

In [2]:
import os
# Set the environment variables for PROJ_LIB and PROJ_DATA
os.environ['PROJ_LIB'] = r'C:\OSGeo4W\share\proj'

from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import psycopg2
from psycopg2 import sql
import geopandas as gpd
import json
from sqlalchemy import create_engine
from pyproj import Transformer
import shapely


#os.environ['PROJ_DATA'] = r'C:\OSGeo4W\share\proj'

# Establish a connection to the database
conn = psycopg2.connect(
    host="localhost",
    database="postgis_sitg",
    user="postgres",
    password="postgres",
    port="5432")

# Set the schema and table names
table_name = "SCANE_INDICE_MOYENNES_3_ANS"
schema_name = "sitg"

nom_rue = 'Rue CAVOUR 20'
annee_idc = '2019'

with conn.cursor() as cur:
    # Get the column names
    cur.execute(f"SELECT * FROM {schema_name}.{table_name} LIMIT 1")
    columns = [desc[0] for desc in cur.description]

    # Get the headers with the applied filters
    cur.execute(f"SELECT * FROM {schema_name}.{table_name} WHERE adresse = %s AND annee = %s", (nom_rue, annee_idc))
    headers = cur.fetchall()

# Print the results
df = pd.DataFrame(headers, columns=columns)

# Ensure 'geometry' is a GeoSeries object
df['geometry'] = gpd.GeoSeries.from_wkb(df['geometry'])

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Set the current CRS for the GeoDataFrame
gdf.set_crs("EPSG:2056", inplace=True)

# convert to WGS84
gdf = gdf.to_crs("EPSG:4326")

# # Transform the coordinates to WGS84 (EPSG:4326)
# gdf['geometry'] = gdf['geometry'].apply(lambda geom: shapely.ops.transform(transformer.transform, geom))

# Calculate centroid and assign to new columns
gdf['coordonnees_rue_lat'] = gdf['geometry'].centroid.y
gdf['coordonnees_rue_lon'] = gdf['geometry'].centroid.x

mean_lat = gdf['coordonnees_rue_lat'].mean()
mean_lon = gdf['coordonnees_rue_lon'].mean()

# convert gdf to geojson
geojson = gdf.__geo_interface__

# convert gdf to pandas dataframe and remove geometry column
df_plot = gdf.drop(columns=['geometry'])

# Plan
fig = px.choropleth_mapbox(df_plot,
                            geojson=geojson,
                            color="indice",
                            locations="egid",
                            featureidkey="properties.egid",
                            center={"lat": mean_lat, "lon": mean_lon},
                            mapbox_style="carto-positron",
                            title="Plan de situation d'indice pour " + str(nom_rue),
                            zoom=17,
                            labels={'adresse': 'npa'})

fig.show()

# close the connection
conn.close()
cur.close()

C:\Users\denis.iglesias\AppData\Local\Temp\ipykernel_16528\4184087208.py:66: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


C:\Users\denis.iglesias\AppData\Local\Temp\ipykernel_16528\4184087208.py:67: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [7]:
conn = psycopg2.connect(
        host="localhost",
        database="postgis_sitg",
        user="postgres",
        password="postgres",
        port="5432")

# Set the schema and table names
table_name = "SCANE_INDICE_MOYENNES_3_ANS"
schema_name = "sitg"

with conn.cursor() as cur:
    # Get the column names
    cur.execute(f"SELECT * FROM {schema_name}.{table_name} LIMIT 1")
    columns = [desc[0] for desc in cur.description]

    # Get the headers with the applied filters
    cur.execute(f"SELECT * FROM {schema_name}.{table_name} WHERE adresse = %s ORDER BY annee ASC", (nom_rue,))
    headers = cur.fetchall()

# Print the results
df_plot2 = pd.DataFrame(headers, columns=columns)
df_plot2 = df_plot2[['annee', 'indice']]
df_plot2['indice'] = df_plot2['indice'].astype(int)
df_plot2['annee'] = df_plot2['annee'].astype(int)

# close the connection
conn.close()
cur.close()

fig2 = px.bar(df_plot2, x='annee', y='indice', text_auto=True)

fig2.update_layout(
    title="Indice de dépense de chaleur [MJ/(m²*an)] le bâtiment situé: " + str(nom_rue),
    xaxis_title="Années",
    yaxis_title="Indice de dépense de chaleur [MJ/(m²*an)]",
    legend_title="Légende",
    xaxis = dict(tickmode = 'linear', dtick = 1),
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    modebar = dict(bgcolor='rgba(0, 0, 0, 0)') # RGB (224,215,255) rgba(0, 0, 0, 0)
    #bargap=0.0
    )
fig2.update_traces(marker=dict(
    color="teal"), #teal
    width = 0.1, #0.2
    textfont_size=12,
    textangle=0,
    textposition="outside",
    cliponaxis=True)
fig2.update_layout(uniformtext_minsize=12, uniformtext_mode='show')
fig2.update_yaxes(visible=False)

fig2.show()



In [20]:
conn = psycopg2.connect(
        host="localhost",
        database="postgis_sitg",
        user="postgres",
        password="postgres",
        port="5432")

# Set the schema and table names
table_name = "SCANE_INDICE_MOYENNES_3_ANS"
schema_name = "sitg"

limite_outlier = 1000

with conn.cursor() as cur:
    # Get the headers with the applied filters
    cur.execute(f"SELECT annee, indice FROM {schema_name}.{table_name} WHERE adresse = %s AND annee = %s", (nom_rue, annee_idc))
    headers_idc = cur.fetchall()

    # Get the headers with the applied filters
    cur.execute(f"SELECT annee, indice, renovation FROM {schema_name}.{table_name} WHERE indice::integer < {limite_outlier} AND annee = %s", (annee_idc,))
    headers_histo = cur.fetchall()

# ligne idc du bâtiment sur histogramme
idc_annee_calcul = pd.DataFrame(headers_idc, columns=['annee', 'indice'])
idc_annee_calcul['indice'] = idc_annee_calcul['indice'].astype(int)
idc_annee_calcul['annee'] = idc_annee_calcul['annee'].astype(int)
idc_annee_calcul = idc_annee_calcul.iloc[0][1]

# data histogramme
df_plot3  = pd.DataFrame(headers_histo, columns=['annee', 'indice', 'renovation'])
df_plot3['indice'] = df_plot3['indice'].astype(int)
df_plot3['annee'] = df_plot3['annee'].astype(int)

# histo
fig3 = px.histogram(df_plot3, x='indice', pattern_shape='renovation', nbins=25, pattern_shape_sequence=["", "/"])
## ligne verticale
#fig3.add_vline(x=idc_annee_calcul, line_dash = 'dash', line_color = 'black', name=nom_rue)
fig3.add_trace(go.Scatter(x=[idc_annee_calcul,idc_annee_calcul],
    y=[25,2500], 
    mode='lines', 
    line=dict(color='black', width=3, dash='dash'),
    name=nom_rue + ' IDC ' + str(annee_idc)))
## rendre joli le truc
fig3.update_layout(
    title="Histogramme des bâtiments du canton " + str(annee_idc),
    xaxis_title="Indice de dépense de chaleur [MJ/(m²*an)]",
    yaxis_title="Nombre de bâtiments",
    legend_title="Légende",
    ### xticks
    xaxis = dict(tickmode = 'linear', dtick = 50),
    ### modifier couleur fond et style
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)",
    modebar = dict(bgcolor='rgba(0, 0, 0, 0)'),
    bargap=0.10)
## couleur de histo
fig3.update_traces(marker=dict(color="teal", line_color="black"))

fig3.show()


In [23]:
conn = psycopg2.connect(
        host="localhost",
        database="postgis_sitg",
        user="postgres",
        password="postgres",
        port="5432")

# Set the schema and table names
table_name = "SCANE_INDICE_MOYENNES_3_ANS"
schema_name = "sitg"

with conn.cursor() as cur:
    # année dropdown
    cur.execute(f"SELECT DISTINCT ANNEE FROM {schema_name}.{table_name} ORDER BY ANNEE ASC")
    results_dropdown_annee = cur.fetchall()

    # Get the headers
    #cur.execute(f"SELECT * FROM {schema_name}.{table_name} WHERE egid::integer = '{egid}' ORDER BY annee ASC")
    cur.execute(f"SELECT DISTINCT ADRESSE FROM {schema_name}.{table_name} ORDER BY ADRESSE ASC")
    results_dropdown_rues = cur.fetchall()

# année dropdown
dropdown_annee = [{"label": i[0], "value": i[0]} for i in results_dropdown_annee]

# rues dropdown
dropdown_rues = [{"label": i[0], "value": i[0]} for i in results_dropdown_rues]

display(dropdown_annee)
display(dropdown_rues)

# Close the connection
cur.close()
conn.close()

[{'label': '2011', 'value': '2011'},
 {'label': '2012', 'value': '2012'},
 {'label': '2013', 'value': '2013'},
 {'label': '2014', 'value': '2014'},
 {'label': '2015', 'value': '2015'},
 {'label': '2016', 'value': '2016'},
 {'label': '2017', 'value': '2017'},
 {'label': '2018', 'value': '2018'},
 {'label': '2019', 'value': '2019'},
 {'label': '2020', 'value': '2020'},
 {'label': '2021', 'value': '2021'},
 {'label': '2022', 'value': '2022'},
 {'label': '2023', 'value': '2023'}]

[{'label': " L'Ancienne-Route 11", 'value': " L'Ancienne-Route 11"},
 {'label': " L'Ancienne-Route 15", 'value': " L'Ancienne-Route 15"},
 {'label': " L'Ancienne-Route 17", 'value': " L'Ancienne-Route 17"},
 {'label': " L'Ancienne-Route 17B", 'value': " L'Ancienne-Route 17B"},
 {'label': " L'Ancienne-Route 19", 'value': " L'Ancienne-Route 19"},
 {'label': " L'Ancienne-Route 20", 'value': " L'Ancienne-Route 20"},
 {'label': " L'Ancienne-Route 20A", 'value': " L'Ancienne-Route 20A"},
 {'label': " L'Ancienne-Route 20B", 'value': " L'Ancienne-Route 20B"},
 {'label': " L'Ancienne-Route 20C", 'value': " L'Ancienne-Route 20C"},
 {'label': " L'Ancienne-Route 20D", 'value': " L'Ancienne-Route 20D"},
 {'label': " L'Ancienne-Route 3", 'value': " L'Ancienne-Route 3"},
 {'label': " L'Ancienne-Route 46", 'value': " L'Ancienne-Route 46"},
 {'label': " L'Ancienne-Route 48", 'value': " L'Ancienne-Route 48"},
 {'label': " L'Ancienne-Route 5", 'value': " L'Ancienne-Route 5"},
 {'label': " L'Ancienne-Rout

In [ ]:
# graphique histogramme général
@app.callback(
    Output('table', 'figure'),
    Input('dropdown_nom_rue', 'value'),
    Input('dropdown_annee_idc', 'value'))

def update_histo(nom_rue, annee_idc):
    conn = psycopg2.connect(
            host="localhost",
            database="postgis_sitg",
            user="postgres",
            password="postgres",
            port="5432")

    # Set the schema and table names
    table_name = "SCANE_INDICE_MOYENNES_3_ANS"
    schema_name = "sitg"

    limite_outlier = 1000

    with conn.cursor() as cur:
        # Get the headers with the applied filters
        cur.execute(f"SELECT * FROM {schema_name}.{table_name} WHERE adresse = %s AND annee = %s LIMIT 1", (nom_rue, annee_idc))
        cols_table = cur.fetchall()

        # Get the headers with the applied filters
        cur.execute(f"SELECT * FROM {schema_name}.{table_name} WHERE adresse = %s AND annee = %s", (nom_rue, annee_idc))
        headers_table = cur.fetchall()

    # dataframe
    table_adresse = pd.DataFrame(headers_table, columns=cols_table)
    
    dash_table.DataTable(data=table_adresse)


